New Topics:

 - Resources for help with coding 
 - Creating a reproducible workflow
 - Merging _pandas_ DataFrames
 - Additional _pandas_ methods 
 

Coding Tasks:

Start a new Jupyter Notebook to complete these tasks. This week, you'll be combining two different datasets.

First, you'll work with a dataset containing the number of primary care physicians per county for each county in the United States. It was obtained from the Area Health Resources File, published by the [Health Resources and Services Administration](https://data.hrsa.gov/topics/health-workforce/ahrf). This data is contained in the file `primary_care_physicians.csv`.

Second, the file `population_by_county.csv` contains the Census Bureau's 2019 population estimates for each US County. It also contains a column `urban`. The `urban` column uses data from the National Bureau of Economic Research to classify each county as either urban or rural. The U.S. Office of Management and Budget designates counties as metropolitan (a core urban area of 50,000 or more population), micropolitan (an urban core of at least 10,000 but less than 50,000 population), or neither. Here, a county is considered "urban" if it is part of a metropolitan or micropolitan area and "rural" if it is not.

 1. First, import the primary care physicians dataset (`primary_care_physicians.csv`) into a data frame named `physicians`. 

In [1]:
import pandas as pd
physicians = pd.read_csv('../data/primary_care_physicians.csv')
physicians.head()

,FIPS,state,county,primary_care_physicians
0,1001,Alabama,Autauga,26.0
1,1003,Alabama,Baldwin,153.0
2,1005,Alabama,Barbour,8.0
3,1007,Alabama,Bibb,12.0
4,1009,Alabama,Blount,12.0


 2. Filter `physicians` down to just the counties in Tennessee. Save the filtered dataframe back to `physicians`. Verify that the resulting dataframe has 95 rows.

In [2]:
physicians = physicians.loc[physicians['state']=='Tennessee']
physicians.shape

(95, 4)

 3. Look at the distribution of the number of primary care physicians. What do you notice?

In [3]:
physicians['primary_care_physicians'].describe()

count     95.000000
mean      51.042105
std      129.311426
min        0.000000
25%        4.500000
50%       12.000000
75%       26.500000
max      806.000000
Name: primary_care_physicians, dtype: float64

The distribution is right skewed, with a large standard deviation. The mean is driven by large hospitals.

 4. Now, import the population by county dataset (`population_by_county.csv`) into a DataFrame named `population`.

In [4]:
population = pd.read_csv('../data/population_by_county.csv')
population.head()

,FIPS,population,county,state,urban
0,17051,21565,Fayette County,ILLINOIS,Rural
1,17107,29003,Logan County,ILLINOIS,Rural
2,17165,23994,Saline County,ILLINOIS,Rural
3,17097,701473,Lake County,ILLINOIS,Urban
4,17127,14219,Massac County,ILLINOIS,Rural


 5. Merge the `physicians` DataFrame with the `population` DataFrame. Keep only the values for Tennessee. When you merge, be sure the include both the `population` and `urban` columns in the merged results. Save the result of the merge back to `physicians`.
 

In [6]:
physicians = pd.merge(left=physicians,right=population[['FIPS','population','urban']],on='FIPS')

In [7]:
physicians

,FIPS,state,county,primary_care_physicians,population,urban
0,47001,Tennessee,Anderson,39.0,76061,Urban
1,47003,Tennessee,Bedford,15.0,48292,Rural
2,47005,Tennessee,Benton,3.0,16140,Rural
3,47007,Tennessee,Bledsoe,1.0,14836,Rural
4,47009,Tennessee,Blount,90.0,129927,Urban
...,...,...,...,...,...,...
90,47181,Tennessee,Wayne,5.0,16693,Rural
91,47183,Tennessee,Weakley,18.0,33510,Rural
92,47185,Tennessee,White,9.0,26800,Rural
93,47187,Tennessee,Williamson,338.0,225389,Urban


 6. How many Tennessee counties are considered urban?

In [10]:
physicians.loc[physicians['urban']=='Urban'].shape
# 38
physicians['urban'].value_counts()

Rural    57
Urban    38
Name: urban, dtype: int64

 7. The State Health Access Data Assistance Center (SHADAC) (https://www.shadac.org/) classifies counties into three groups based on the number of residents per primary care physician. First, counties with fewer than 1500 residents per primary care physician are considered to have an "adequate" supply. Counties with at least 1500 residents but fewer than 3500 residents per primary care physician are considered to have a "moderately inadequate" supply, and counties with at least 3500 residents per primary care physician are considered to have a "low inadequate" supply. How many counties in Tennessee are in each group?

In [28]:
physicians.loc[(physicians['population'] < 1500*physicians['primary_care_physicians'])]['county'].count()
# 14 adequate

14

In [29]:
physicians.loc[(physicians['population'] >= 1500*physicians['primary_care_physicians'])& (physicians['population'] <3500*physicians['primary_care_physicians'])]['county'].count()
# 50 moderately inadequate

50

In [30]:
physicians.loc[(physicians['population'] >= 3500*physicians['primary_care_physicians'])]['county'].count()
# 31 low inadequat

31

  8. Does there appear to be any detectable relationship between whether a county is urban or rural and its supply of primary care physicians?

In [34]:
physicians.loc[(physicians['population'] < 1500*physicians['primary_care_physicians'])]['urban'].describe()

count        14
unique        2
top       Urban
freq          9
Name: urban, dtype: object

In [35]:
physicians.loc[(physicians['population'] >= 1500*physicians['primary_care_physicians'])& (physicians['population'] <3500*physicians['primary_care_physicians'])]['urban'].describe()

count        50
unique        2
top       Rural
freq         31
Name: urban, dtype: object

In [36]:
physicians.loc[(physicians['population'] >= 3500*physicians['primary_care_physicians'])]['urban'].describe()

count        31
unique        2
top       Rural
freq         21
Name: urban, dtype: object

In [ ]:
# urban counties are more likely to be adequate than rural counties.